# Repositorio PUCP Posgrado

In [18]:
###-Repositorio estadistica informatica pregrado UNALM-### 

#Trabajo Final → AVANCE.
#1. Relación de repositorios con sus respectivos url
#2. Automatización de la descarga de la información de un repositorio.
#3. Estructuración de la información descargada en formato csv

url = 'https://tesis.pucp.edu.pe/repositorio/handle/20.500.12404/757/recent-submissions'

#paquetes 
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import re as re 
import pandas as pd

import ssl
context = ssl._create_unverified_context() #la pagina pide un socket seguro 

#Las tesis se encuentran en 4 paginas distinas 

#devuelve la url de la siguiente pagina 
def next_page(url):
    page = urllib.request.urlopen(url,context = context)
    soup = BeautifulSoup(page, "html.parser")
    
    line = soup.find('a',{'class':'next-page-link'})
    line = line.decode().strip()
    offset = re.findall(r'\?offset=.{2}',line)
    url_diff = offset[0]
    
    url = re.sub('\?offset.*','',url)
    n_url = url + url_diff
    return n_url

URL = []
url1 = url

#devuelve las url de todas las paginas 
def all_pages(url,URL):
    
    page = urllib.request.urlopen(url,context = context)
    soup = BeautifulSoup(page, "html.parser")
    last = soup.find('li',{'class':'next pull-right disabled'})
    URL.append(url)
    if last == None: 
        url = next_page(url)
        all_pages(url,URL)
    else: 
        return URL 
    return URL 
all_pages(url1,URL)

###Almacenar links de las tesis 

handles = [] 

#las url de las tesis se diferencian por el codigo handle/20.500.12404/***** <-- diferencia 
for urls in URL: 
    page = urllib.request.urlopen(urls,context = context)
    soup = BeautifulSoup(page, "html.parser")
    for lines in soup.find_all('h4'):
        lines = lines.decode().strip()
        link = re.findall(r'(?<=20.500.12404/).+?(?=\")',lines) #handle unico a cada tesis (#12996)
        handles.append(link)
       
## Links completos 

#creando las url especificas de cada tesis 
links=[]
def get_links(handles,links):
    handle_url = re.sub('(?<=20.500.12404/).+','',url)
    for i in range(len(handles)): 
        links.append(handle_url+handles[i][0])
    return links

get_links(handles,links)    

#agregando ?show=full a las url se muestra informacion mas detallada de cada tesis 
links_full = [] 
for link in links: 
    links_full.append(link+'?show=full')
    

## almacenando en diccionario 

rep_PUCP = { 
        'unis' : [],
        'titulos' : [],
        'nombres' : [],
        'grados' : [], 
        'asesores' : [],
        'resumenes': [],
        'años' : []}
 
def get_all(links_full,reps_PUCP):
    for i in range(len(links_full)): 
        page = urllib.request.urlopen(links_full[i],context = context)
        soup = BeautifulSoup(page, "html.parser")
        full_t=''
        for line in soup.find_all('tr'):
            full_t = line.decode().strip()+'\n'+full_t
        
        uni = re.findall(r'(?<=>thesis\.degree\.grantor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        titulo = re.findall(r'(?<=>dc\.title</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        nombre = re.findall(r'(?<=>dc\.contributor\.author</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        grado = re.findall(r'(?<=>thesis\.degree\.name</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        asesor = re.findall(r'(?<=>dc\.contributor\.advisor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        resumen = re.findall(r'(?<=<tr class=\"ds-table-row even\">\n<td class=\"label-cell\">dc\.description\.abstract</td><td class=\"word-break\">)[\s\S]*?(?=</td><td>)',full_t)
        año = re.findall(r'(?<=>dc\.date\.issued</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        
        temas_prompt = r'(?<=>dc.subject</td><td class=\"word-break\">).+?(?=</td>)'
        temas_match = re.findall(temas_prompt,full_t)
        print(temas_match)
        
        
        
        rep_PUCP['unis'].append(uni[0])
        rep_PUCP['titulos'].append(titulo[0])     
        rep_PUCP['nombres'].append(nombre[0])
        rep_PUCP['grados'].append(grado[0]) 
        
        if asesor == []: 
            rep_PUCP['asesores'].append('Sin asesor')
        else:
            rep_PUCP['asesores'].append(asesor[0])
        if resumen == []: 
            rep_PUCP['resumenes'].append('Sin resumen')
        else:
            rep_PUCP['resumenes'].append(resumen[0])
        rep_PUCP['años'].append(año[0])
    return reps_PUCP

PUCP = get_all(links_full,rep_PUCP)

['Indicadores financieros--Análisis', 'Sistema financiero--Perú', 'Análisis financiero--Perú']
['Variables (Estadística)', 'Estadística bayesiana', 'Modelos lineales (Estadística)']
['Variables (Estadística)', 'Procesamiento de datos', 'Redes neuronales']
['Inferencia estadística', 'Análisis de regresión']
['Estadística matemática', 'Seguros de automóviles--Modelos matemáticos', 'Tarifas--Modelos matemáticos']
['Variables latentes', 'Estadística bayesiana']
['Método de Monte Carlo', 'Análisis de regresión', 'Estadística bayesiana']
['Variables latentes', 'Estadística bayesiana']
['Estimación de parámetros', 'Estadística bayesiana', 'Algoritmos']
['Modelos lineales (Estadística)', 'Análisis de regresión', 'Estadística bayesiana']
['Análisis de regresión', 'Variables (Estadística)', 'Estadística bayesiana']
['Modelos lineales (Estadística)', 'Análisis de regresión']
['Análisis de regresión', 'Estadística--Modelos matemáticos', 'Estadística bayesiana']
['Estadística bayesiana', 'Modelos l

In [19]:
df_PUCP = pd.DataFrame(PUCP)

# Repositorio UNALM Pregrado

In [20]:
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import re as re 
import pandas as pd

url = 'https://repositorio.lamolina.edu.pe/handle/20.500.12996/26/recent-submissions'

# url de la siguiente pagina
def next_page(url):
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    
    line = soup.find('a',{'class':'next-page-link'})
    line = line.decode().strip()
    offset = re.findall(r'\?offset=.{2}',line)
    url_diff = offset[0]
    
    url = re.sub('\?offset.*','',url)
    n_url = url + url_diff
    return n_url

#Se extrae todos los url de todas las paginas 
URL = []
url1 = url

def all_pages(url,URL):
    
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    last = soup.find('li',{'class':'next pull-right disabled'})
    URL.append(url)
    if last == None: 
        url = next_page(url)
        all_pages(url,URL)
    else: 
        return URL 
    return URL 
all_pages(url1,URL)

###Almacenar links de las tesis 

handles = [] 
#Se extraen los urls de todas las tesis 
for urls in URL: 
    page = urllib.request.urlopen(urls)
    soup = BeautifulSoup(page, "html.parser")
    for lines in soup.find_all('h4'):
        lines = lines.decode().strip()
        link = re.findall(r'(?<=20.500.12996/).+?(?=\")',lines) #handle unico a cada tesis 
        handles.append(link)

## Links completos 

links=[]
def get_links(handles,links):
    handle_url = re.sub('(?<=20.500.12996/).+','',url)
    for i in range(len(handles)): 
        links.append(handle_url+handles[i][0])
    return links

get_links(handles,links)    

links_full = [] 
for link in links: 
    links_full.append(link+'?show=full')

# Dicionario de repositorio UNALM
rep_unalm = { 
        'unis' : [],
        'titulos' : [],
        'nombres' : [],
        'grados' : [], 
        'asesores' : [],
        'resumenes': [],
        'años' : [] }
        
# Se define una función para obtener el nombre de la institución, el título de la tesis, el nombre del tesista, el grado al que aspira, su asesor, el resumen de la tesis y el año en que se publicó.
def get_all(links_full,reps_unalm):
    for i in range(len(links_full)): 
        page = urllib.request.urlopen(links_full[i])
        soup = BeautifulSoup(page, "html.parser")
        full_t=''
        for line in soup.find_all('tr'):
            full_t = line.decode().strip()+'\n'+full_t
        #se hace uso de regex para encontrar todo lo pedido    
        uni = re.findall(r'(?<=>thesis\.degree\.grantor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        titulo = re.findall(r'(?<=>dc\.title</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        nombre = re.findall(r'(?<=>dc\.contributor\.author</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        grado = re.findall(r'(?<=>thesis\.degree\.name</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        asesor = re.findall(r'(?<=>dc\.contributor\.advisor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        resumen = re.findall(r'(?<=>dc\.description\.abstract</td><td class=\"word-break\">).+?(?=</td><td>es_PE</td>)',full_t)
        año = re.findall(r'(?<=>dc\.date\.accessioned</td><td class=\"word-break\">).+?(?=T)',full_t)
        
        temas_prompt = r'(?<=>dc.subject</td><td class=\"word-break\">).+?(?=</td>)'
        temas_match = re.findall(temas_prompt,full_t)
        print(temas_match)
        
        rep_unalm['unis'].append(uni[0])
        rep_unalm['titulos'].append(titulo[0])     
        rep_unalm['nombres'].append(nombre[0])
        rep_unalm['grados'].append(grado[0]) 
        
        if asesor == []: 
            rep_unalm['asesores'].append('N/A')
        else:
            rep_unalm['asesores'].append(asesor[0])
            
        rep_unalm['resumenes'].append(resumen[0])
        rep_unalm['años'].append(año[0])
    return reps_unalm

UNALM = get_all(links_full,rep_unalm)

['Consumidores', 'Modelos de simulación', 'Riesgo', 'Empresas privadas', 'Compañías de seguros', 'Estrategias de control', 'Seguros', 'Accidentes', 'Siniestros', 'Vehículos']
['Política monetaria', 'Perú', 'Métodos estadísticos', 'Lima Metropolitana', 'Instituciones financieras', 'Desarrollo económico', 'Método Stacking']
['Perú', 'Gestión', 'Rentabilidad', 'Soporte de decision', 'Toma de decisiones', 'Hospitales']
['Algoritmo Random Forest', 'Regresión logística', 'Riesgo de fuga', 'Fuga de clientes', 'Perú', 'Evaluación', 'Modelos matemáticos', 'Métodos estadísticos', 'Análisis de datos', 'Análisis de la regresión', 'Técnicas de predicción', 'Comportamiento del consumidor', 'Consumidores', 'Empresas', 'Telecomunicaciones']
['Compañías de seguros', 'SOAT', 'Perú', 'Evaluación', 'Modelos de simulación', 'Riesgo', 'Costos', 'Seguros', 'Análisis de coste', 'Siniestros', 'Double Chain Ladder']
['Aplicativos', 'Bases de datos', 'Registro', 'Estrategias', 'Operaciones', 'Planificación']
['B

In [ ]:
df_UNALM = pd.DataFrame(UNALM)


# Repositorio UNALM Posgrado

In [13]:
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import re as re 
import pandas as pd

url = 'http://repositorio.lamolina.edu.pe/handle/20.500.12996/329'

# Se importa las librerias necesarias
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import re as re 


#Se extrae todos los url de todas las paginas 
URL = []

def all_pages(url,URL):
    
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    last = soup.find('li',{'class':'next pull-right disabled'})
    URL.append(url)
    return URL 
all_pages(url,URL)

###Almacenar links de las tesis 

handles = [] 
#Se ectraen los urls de todas las tesis 
for urls in URL: 
    page = urllib.request.urlopen(urls)
    soup = BeautifulSoup(page, "html.parser")
    for lines in soup.find_all('h4'):
        lines = lines.decode().strip()
        link = re.findall(r'(?<=20.500.12996/).+?(?=\")',lines) #handle unico a cada tesis 
        handles.append(link)

## Links completos 

links=[]
def get_links(handles,links):
    handle_url = re.sub('(?<=20.500.12996/).+','',url)
    for i in range(len(handles)): 
        links.append(handle_url+handles[i][0])
    return links

get_links(handles,links)    

links_full = [] 
for link in links: 
    links_full.append(link+'?show=full')

    

# Dicionario de repositorio UNALM
rep_unalmp = { 
        'unis' : [],
        'titulos' : [],
        'nombres' : [],
        'grados' : [], 
        'asesores' : [],
        'resumenes': [],
        'años' : [] }
        
# Se define una función para obtener el nombre de la institución, el título de la tesis, el nombre del tesista, el grado al que aspira, su asesor, el resumen de la tesis y el año en que se publicó.
def get_all(links_full,reps_unalmp):
    for i in range(len(links_full)): 
        page = urllib.request.urlopen(links_full[i])
        soup = BeautifulSoup(page, "html.parser")
        full_t=''
        for line in soup.find_all('tr'):
            full_t = line.decode().strip()+'\n'+full_t
        #se hace uso de regex para encontrar todo lo pedido    
        uni = re.findall(r'(?<=>thesis\.degree\.grantor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        titulo = re.findall(r'(?<=>dc\.title</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        nombre = re.findall(r'(?<=>dc\.contributor\.author</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        grado = re.findall(r'(?<=>thesis\.degree\.name</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        asesor = re.findall(r'(?<=>dc\.contributor\.advisor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        resumen = re.findall(r'(?<=>dc\.description\.abstract</td><td class=\"word-break\">).+?(?=</td><td>es_PE</td>)',full_t)
        año = re.findall(r'(?<=>dc\.date\.accessioned</td><td class=\"word-break\">).+?(?=T)',full_t)
        
        temas_prompt = r'(?<=>dc.subject</td><td class=\"word-break\">).+?(?=</td>)'
        temas_match = re.findall(temas_prompt,full_t)
        print(temas_match)
        
        rep_unalmp['unis'].append(uni[0])
        rep_unalmp['titulos'].append(titulo[0])     
        rep_unalmp['nombres'].append(nombre[0])
        rep_unalmp['grados'].append(grado[0]) 
        
        if asesor == []: 
            rep_unalmp['asesores'].append('N/A')
        else:
            rep_unalmp['asesores'].append(asesor[0])
            
        rep_unalmp['resumenes'].append(resumen[0])
        rep_unalmp['años'].append(año[0])
    return reps_unalmp

UNALMp = get_all(links_full,rep_unalmp)

['Análisis de conglomerados', 'Eficiencia económica', 'Perú', 'Comunicación', 'Transporte', 'Salud', 'Educación', 'Datos globales', 'Indicadores económicos', 'Gastos públicos', 'Gobierno regional']
['Universidades', 'Perú', 'Evaluación', 'Métodos estadísticos', 'Medición', 'Grupos de interés', 'Comportamiento del consumidor', 'Encuestas al consumidor', 'Satisfacción del alumno', 'Calidad', 'Servicio al cliente']
['Modelos econométricos', 'Modelo de ecuaciones estructurales', 'Métodos estadísticos', 'Métodos de estimación de mínimos cuadrados parciales', 'Análisis de datos', 'Recursos humanos', 'Productividad del trabajo', 'Calidad de vida', 'Violencia familiar', 'Perú', 'Mujeres']
['Modelos de ecuaciones']
['Estadística']
['Método de equipercentil', 'Método lineal', 'Equiparación de puntajes', 'Examen de admisión', 'Perú', 'Evaluación', 'Métodos estadísticos', 'Universidades', 'Estudiantes']
['Universidad Nacional Agraria La Molina', 'Rendimiento académico', 'Estadística general', 'Per

In [ ]:
df_UNALMp = pd.DataFrame(UNALMp)


In [ ]:
#se une todas las informaciones obtenidas
Tesis = pd.concat([df_PUCP, df_UNALM, df_UNALMp], axis=0)
#Tesis
Tesis.to_csv('Tesis.csv', index=False,encoding='utf-8-sig',
          header = ['Institución','Título de la tesis', 'Nombre del tesista', 'Grado', 'Nombre del asesor', 'Resumen','Año'])
df_Tesis_read = pd.read_csv('Tesis.csv')
